## What's New

### Summary 

Brought to you by popular demand, this edition of `made-with-gs-quant` will showcase some of our newest features and capabilities. Going forward, look out for a `What's New` section in `made_with_gs_quant` that will highlight new additions.

The content of this notebook is split into:
* [1 - Let's get started with gs quant](#1---Let's-get-started-with-gs-quant)
* [2 - Cash flows](#2---Cash-flows)
* [3 - Portfolios](#3---Portfolios)
* [4 - Scenarios](#4---Scenarios)
* [5 - Compute](#4---Compute)

### 1 - Let's get started with gs quant
Start every session with authenticating with your unique client id and secret. If you don't have a registered app, create one [here](https://marquee.gs.com/s/developer/myapps/register). `run_analytics` scope is required for the functionality covered in this example. Below produced using gs-quant version 0.8.115.

In [1]:
from gs_quant.session import GsSession
GsSession.use(client_id=None, client_secret=None, scopes=('run_analytics',))

### 2 - Cash flows

Let's start with something small but impactful - ability to model future cashflows. You can simply use `CashFlows` as any of the other risk measures to get a dataframe of future cashflows.

In [2]:
from gs_quant.risk import Cashflows
from gs_quant.common import Currency, PayReceive
from gs_quant.instrument import IRSwaption

swaption = IRSwaption(PayReceive.Receive, '10y', Currency.USD, expiration_date='6m')
cf = swaption.calc(Cashflows)
cf.head()

,payment_date,payment_type,accrual_start_date,accrual_end_date,floating_rate_option,floating_rate_designated_maturity,rate,payment_amount,spread
0,2021-01-15,Flt,2020-10-15,2021-01-15,USD-LIBOR-BBA,3m,0.003841,98171.653675,0.0
1,2021-04-15,Fix,2020-10-15,2021-04-15,None,None,0.008309,-415454.698778,NaN
2,2021-04-15,Flt,2021-01-15,2021-04-15,USD-LIBOR-BBA,3m,0.003345,83618.720155,0.0
3,2021-07-15,Flt,2021-04-15,2021-07-15,USD-LIBOR-BBA,3m,0.002895,73184.073125,0.0
4,2021-10-15,Fix,2021-04-15,2021-10-15,None,None,0.008309,-415454.698778,NaN


### 3 - Portfolios
I have used `Portfolio` wrappers in my previous analyses, so this one should be somewhat familiar. `Portfolio` makes working with many positions easier and allows risk to be viewed both at position level and aggregate level (more portfolio features like saving in the pipeline..look for it in `What's New` section in the new editions!)

Let's first define a portfolio - I will define a book of swaptions with varying strikes.

In [3]:
from gs_quant.markets.portfolio import Portfolio
from gs_quant.common import Currency, PayReceive
from gs_quant.instrument import IRSwaption

swaptions = [IRSwaption(PayReceive.Receive, '10y', Currency.USD, expiration_date='6m', strike=strike, 
                        name=strike) for strike in ('atm', 'atm-10', 'atm-25', 'atm-50')]

# create portfolio from list of instruments
portfolio = Portfolio(swaptions)

# resolve all the instruments in place
portfolio.resolve()

Now we can work with this portfolio the same way we do with an instrument - that is, we can price it or calculate any of the available risk measures on it. 

In [4]:
from gs_quant.risk import DollarPrice, IRVegaParallel
risks = portfolio.calc((DollarPrice, IRVegaParallel))

With `risks` in hand, we can examine these risks on an instrument level or aggregate level:

In [5]:
# all risks for a specific (in this case, first) instrument in the portfolio
print(risks[portfolio[0]])

# single risk for a specific (in this case, first) instrument in the portfolio
print(risks[portfolio[0]][DollarPrice])
# or...
print(risks[DollarPrice][portfolio[0]])

# aggregate risk for the portfolio
print(risks[IRVegaParallel].aggregate())

{DollarPrice: 2280775.863350406, IRVegaParallel: 27563.74301610279}
2280775.863350406
2280775.863350406
94937.17103650255


Finally, we can convert our portfolio to a dataframe where each row will represent an instrument. 

In [6]:
frame = portfolio.to_frame()
frame.head(2)

,notional_currency,fee_payment_date,effective_date,asset_class,floating_rate_frequency,floating_rate_option,strike,pay_or_receive,buy_sell,expiration_date,...,type,premium_payment_date,termination_date,floating_rate_day_count_fraction,fee,floating_rate_designated_maturity,premium,fee_currency,settlement,fixed_rate_day_count_fraction
instrument,,,,,,,,,,,,,,,,,,,,,
atm,Currency.USD,2020-04-15,2020-10-15,AssetClass.Rates,3m,USD-LIBOR-BBA,0.008309,Receive,BuySell.Buy,2020-10-13,...,AssetType.Swaption,2020-04-15,2030-10-15,DayCountFraction.ACT_OVER_360,0.0,3m,0.0,Currency.USD,SwapSettlement.Phys_CLEARED,DayCountFraction._30_OVER_360
atm-10,Currency.USD,2020-04-15,2020-10-15,AssetClass.Rates,3m,USD-LIBOR-BBA,0.007309,Receive,BuySell.Buy,2020-10-13,...,AssetType.Swaption,2020-04-15,2030-10-15,DayCountFraction.ACT_OVER_360,0.0,3m,0.0,Currency.USD,SwapSettlement.Phys_CLEARED,DayCountFraction._30_OVER_360


We can append `RiskResults` as columns to this dataframe as we compute additional measures.

In [7]:
import pandas as pd

rks = pd.DataFrame(risks) 
rks.index = frame.index

pd.concat([frame, rks], axis=1)

,notional_currency,fee_payment_date,effective_date,asset_class,floating_rate_frequency,floating_rate_option,strike,pay_or_receive,buy_sell,expiration_date,...,termination_date,floating_rate_day_count_fraction,fee,floating_rate_designated_maturity,premium,fee_currency,settlement,fixed_rate_day_count_fraction,DollarPrice,IRVegaParallel
instrument,,,,,,,,,,,,,,,,,,,,,
atm,Currency.USD,2020-04-15,2020-10-15,AssetClass.Rates,3m,USD-LIBOR-BBA,0.008309,Receive,BuySell.Buy,2020-10-13,...,2030-10-15,DayCountFraction.ACT_OVER_360,0.0,3m,0.0,Currency.USD,SwapSettlement.Phys_CLEARED,DayCountFraction._30_OVER_360,2.280776e+06,27563.743016
atm-10,Currency.USD,2020-04-15,2020-10-15,AssetClass.Rates,3m,USD-LIBOR-BBA,0.007309,Receive,BuySell.Buy,2020-10-13,...,2030-10-15,DayCountFraction.ACT_OVER_360,0.0,3m,0.0,Currency.USD,SwapSettlement.Phys_CLEARED,DayCountFraction._30_OVER_360,1.784680e+06,26885.215492
atm-25,Currency.USD,2020-04-15,2020-10-15,AssetClass.Rates,3m,USD-LIBOR-BBA,0.005809,Receive,BuySell.Buy,2020-10-13,...,2030-10-15,DayCountFraction.ACT_OVER_360,0.0,3m,0.0,Currency.USD,SwapSettlement.Phys_CLEARED,DayCountFraction._30_OVER_360,1.184394e+06,24093.522927
atm-50,Currency.USD,2020-04-15,2020-10-15,AssetClass.Rates,3m,USD-LIBOR-BBA,0.003309,Receive,BuySell.Buy,2020-10-13,...,2030-10-15,DayCountFraction.ACT_OVER_360,0.0,3m,0.0,Currency.USD,SwapSettlement.Phys_CLEARED,DayCountFraction._30_OVER_360,5.390310e+05,16394.689601


Note the last two columns in the frame are the risks we appended!

Want to see more portfolio examples? Refer to portfolio [examples here](https://nbviewer.jupyter.org/github/goldmansachs/gs-quant/tree/master/gs_quant/examples/02_portfolio/) and [previous editions](https://nbviewer.jupyter.org/github/goldmansachs/gs-quant/tree/master/gs_quant/made_with_gs_quant/) of `made_with_gs_quant`.

### 4 - Scenarios

The team has also invested to build our scenario capabilities. In particular we: (1) simplified `MarketDataShock` scenario (2) added aging or `carry` scenario (3) introduced composite scenarios. Let's look at an example of each.

##### Simple Market Shock

`MarketDataShockBasedScenario` is designed to allow users to shock any kind of market data. In this example we will shock the spot by 1bp. Note while before you had to specify each of the underlying curves used to build spot curve, you can now simply indicate `('IR', 'USD')`

In [8]:
from gs_quant.risk import MarketDataShockBasedScenario, MarketDataPattern, MarketDataShock, MarketDataShockType

ir_spot_scenario = MarketDataShockBasedScenario(
    shocks={
        MarketDataPattern('IR', 'USD'): MarketDataShock(MarketDataShockType.Absolute, 1e-4),  # 1 bp shock
        MarketDataPattern('IR Reset', 'USD-3m'): MarketDataShock(shock_type=MarketDataShockType.Absolute, value=1e-4),
    })

with ir_spot_scenario:
    spot_shock = portfolio.price()
    
print('Base price:        {:,.2f}'.format(risks[DollarPrice].aggregate())) # or portfolio.price().aggregate()
print('Spot shock price:  {:,.2f}'.format(spot_shock.aggregate()))

Base price:        5,788,880.72
Spot shock price:  5,668,925.63


##### Carry Scenario

The `CarryScenario` can be used to move the pricing date into the future. In order to do this we do two things: (1) project the market data forward in time (either keeping spot rates constant or fwd rates constant - more on this in a bit) to the new pricing date and (2) lifecycle the trades so that any fixings or expiries that happen between today’s date and the target future date have been handled.

In [9]:
from gs_quant.markets import PricingContext
from gs_quant.risk import CarryScenario

carry_scenario = CarryScenario(time_shift=22)
with carry_scenario:
    carry_shock = portfolio.price()
    
print('Base price:        {:,.2f}'.format(risks[DollarPrice].aggregate())) # portfolio.price().aggregate()
print('Carry shock price: {:,.2f}'.format(carry_shock.aggregate()))

Base price:        5,788,880.72
Carry shock price: 5,156,405.53


##### Combined Scenarios

We can now combine these two scenarios to produce a 'carry then bump' scenario (note all permutations aren't supported yet but are underway):

In [10]:
with carry_scenario, ir_spot_scenario:
    carry_then_bump_shock = portfolio.price()
    
print('Base price:            {:,.2f}'.format(risks[DollarPrice].aggregate())) # portfolio.price().aggregate()
print('Carry then bump price: {:,.2f}'.format(carry_then_bump_shock.aggregate()))

Base price:            5,788,880.72
Carry then bump price: 4,999,333.03


More on scenario examples in the [examples folder](https://nbviewer.jupyter.org/github/goldmansachs/gs-quant/tree/master/gs_quant/examples/04_scenario/) in the gs-quant repo.

### 5 - Compute

Finally to optimize both speed and performance we have improved our async and batch framework.

In a nutshell, `async` allows the requests to be processed in parallel and `batch` allows them to be grouped together and avoid timeouts for particularly large requests.

Additionally, multiple contexts can be controlled by a 'master' `PricingContext` to batch requests together in order to also avoid timeouts and sending too many requests. For example of this please see [`4-Delta Hedging`](https://nbviewer.jupyter.org/github/goldmansachs/gs-quant/blob/master/gs_quant/made_with_gs_quant/4-Delta%20Hedging.ipynb) notebook. 

For now, let's look at a simpler example using the scenarios we discussed in the previous section. Here we will look at how the price of our portfolio evolved as we move it 21 days forward in time both keeping spot rates constant (`roll_to_fwds=False`) and fwd rates constant (`roll_to_fwds=True`). Let's use a `PricingContext` with batch mode to organize these requests.

In [11]:
from gs_quant.markets import PricingContext

roll_to_spot = []
roll_to_fwd = []

with PricingContext(is_batch=True):
    for time_shift in range(22):
        with PricingContext(is_async=True), CarryScenario(time_shift, roll_to_fwds=True):
            roll_to_spot.append(portfolio.price())
        with PricingContext(is_async=True), CarryScenario(time_shift, roll_to_fwds=False):
            roll_to_fwd.append(portfolio.price())

Let's now visualize the result:

In [12]:
import matplotlib.pyplot as plt

pd.Series([r.aggregate() for r in roll_to_spot], name='Roll to Spot', dtype=np.dtype(float)).plot(figsize=(10, 6))
pd.Series([r.aggregate() for r in roll_to_fwd], name='Roll to Fwd', dtype=np.dtype(float)).plot(figsize=(10, 6))
plt.xlabel('TimeShift')
plt.ylabel('PV')

Text(0, 0.5, 'PV')

That's all for this edition - please keep sharing your feedback on what would make `gs-quant` even more useful for you as well as suggestions for `made_with_gs_quant` you would like to see in the future.

Happy coding!